Socioeconomic factors affecting fertility 
=================================

# Introduction

The purpose of this analysis is to identify some of the socioeconomic variables that may be connected to the fertility rate of each country. The dataset is provided by the World Bank and is hosted in BigQuery. More information can be found [here](https://www.kaggle.com/theworldbank/world-bank-health-population).

Health statistics for different countries and years are included in the dataset, and the total fertility rate is the main variable of interest in this analysis. This rate generally tracks the number of children an average woman will have during her reproductive years. A more detailed definition is provided by the United Nations as:

"The average number of live births a hypothetical cohort of women would have at the end of their reproductive period if they were subject during their whole lives to the fertility rates of a given period and if they were not subject to mortality. It is expressed as live births per woman."


According to the United Nations, a level of  2.1 children per woman is required in most countries to maintain a stable population. This is also called "replacement level fertility" and may be higher for countries with high infant mortality rates.

# BigQuery client

In [1]:
from google.cloud import bigquery
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [2]:
# Create a "Client" object
client = bigquery.Client()

Using Kaggle's public dataset BigQuery integration.


In [3]:
# Reference to the dataset
dataset_ref = client.dataset('world_bank_health_population', project='bigquery-public-data')

# Fetch data
dataset = client.get_dataset(dataset_ref)

# List all the tables in the dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

country_series_definitions
country_summary
health_nutrition_population
series_summary
series_times


In [4]:
# Reference to the 'crime_by_lsoa' table
table_ref = dataset_ref.table('health_nutrition_population')

# Fetch the table
table = client.get_table(table_ref)

# Dataset
client.list_rows(table, max_results = 10).to_dataframe()

,country_name,country_code,indicator_name,indicator_code,value,year
0,Mexico,MEX,"School enrollment, primary, male (% gross)",SE.PRM.ENRR.MA,109.898956,2007
1,Mexico,MEX,"School enrollment, secondary (% gross)",SE.SEC.ENRR,97.343384,2014
2,Mexico,MEX,"School enrollment, secondary, female (% gross)",SE.SEC.ENRR.FE,56.050011,1987
3,Mexico,MEX,"School enrollment, secondary, female (% gross)",SE.SEC.ENRR.FE,87.523102,2007
4,Mexico,MEX,"School enrollment, secondary, female (% net)",SE.SEC.NENR.FE,65.421370,2004
5,Mexico,MEX,"School enrollment, tertiary, female (% gross)",SE.TER.ENRR.FE,10.506150,1983
6,Mexico,MEX,Sex ratio at birth (male births per female bir...,SP.POP.BRTH.MF,1.050000,1987
7,Mexico,MEX,Sex ratio at birth (male births per female bir...,SP.POP.BRTH.MF,1.050000,1992
8,Mexico,MEX,Share of women employed in the nonagricultural...,SL.EMP.INSV.FE.ZS,37.500000,2002
9,Mexico,MEX,"Suicide mortality rate, female (per 100,000 fe...",SH.STA.SUIC.FE.P5,2.300000,2016


# 1. Impact of education and income on fertility

The main variable used in this section is the gross enrollment ratio of female students in secondary and tertiary education. As defined by the World Bank:

"Gross enrollment ratio is the ratio of total enrollment, regardless of age, to the population of the age group that officially corresponds to the level of education shown."

Therefore, this indicator may be higher than normal, or even higher than 100%, when a large number of overage or underage students exists.

In addition, the Gross National Income (GNI) per capita (PPP) in current international dollars is added into the analysis to give more depth. In a simple sense, this indicator measures the total income earned by the residents of a country, divided by the population. An important detail is the purchasing power parity (PPP) conversion. This implies that the differences between the price levels are taken into account, in order to enable meaningful comparisons between countries.

In [5]:
query1 = """

    SELECT  country_name, indicator_name, value, year, region
            FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` AS indices
            JOIN `bigquery-public-data.world_bank_health_population.country_summary` AS regions
                ON indices.country_code = regions.country_code
            WHERE indicator_code IN ('SP.DYN.TFRT.IN', 'SE.SEC.ENRR.FE', 'SE.TER.ENRR.FE')
            ORDER BY country_name, indicator_name, year
            ;
        """

In [6]:
education = client.query(query1).result().to_dataframe()

In [7]:
# Secondary query to get gross national income data from the World Bank education dataset
query1_2 = """

    SELECT  country_name, indicator_name, value, year, region
            FROM `bigquery-public-data.world_bank_intl_education.international_education` indices
            JOIN `bigquery-public-data.world_bank_intl_education.country_summary` AS regions
                ON indices.country_code = regions.country_code
            WHERE indicator_code = 'NY.GNP.PCAP.PP.CD'
            ORDER BY country_name, year
            ;
        """

In [8]:
income = client.query(query1_2).result().to_dataframe()

In [9]:
# Dataframes for scatter plots

# Combine tables
education_income = pd.concat([education, income])

# Sort
education_income = education_income.sort_values(by=['region','country_name', 'indicator_name', 'year'])

# Pivot
education_income = education_income.pivot(values='value', index=['country_name','year','region'], 
                                                columns=['indicator_name'])

education_income = education_income.sort_values(by=['region','country_name', 'year'])
education_income = education_income.reset_index()

# Rename columns
education_income.set_axis(['country', 'year', 'region', 'fertility', 'gni', 'secondary', 'tertiary'], axis='columns', 
                                                                                                        inplace=True)
# Cleansing
education_income = education_income[(education_income['year']>=1990) & (education_income['year']<=2016 )]
education_income = education_income.fillna(0)
education_income = education_income[education_income['region']!=0]

secondary_income = tertiary_income = education_income

secondary_income.loc[secondary_income['gni'] == 0, ['fertility', 'secondary']] = 0
secondary_income.loc[secondary_income['fertility'] == 0, ['gni', 'secondary']] = 0
secondary_income.loc[secondary_income['secondary'] == 0, ['fertility', 'gni']] = 0

tertiary_income.loc[tertiary_income['gni'] == 0, ['fertility', 'tertiary']] = 0
tertiary_income.loc[tertiary_income['fertility'] == 0, ['gni', 'tertiary']] = 0
tertiary_income.loc[tertiary_income['tertiary'] == 0, ['fertility', 'gni']] = 0

In [10]:
# Scatter plot secondary education
fig = px.scatter(secondary_income, x='fertility', y='secondary', animation_frame='year', animation_group='country',
        size='gni', color='region', hover_name='country',
        labels={ 
           'fertility': 'Fertility rate',
           'secondary': 'School enrollment, secondary, female %',
           'gni':'GNI per capita PPP'
        },
        title='Impact of secondary education and income on fertility',
        range_x=[0.3,8], range_y=[1,170])

fig.add_annotation(dict(font=dict(size=13), x=0, y=-0.12, showarrow=False,
                                    text='Size is GNI per capita PPP',
                                    textangle=0,
                                    xanchor='left',
                                    xref='paper', yref='paper'))

fig.layout.updatemenus[0].buttons[0]['visible'] = False
fig.layout.updatemenus[0].buttons[1]['visible'] = False
fig.show()

European countries seem to have the lowest fertility rates and a high degree of secondary school enrollment during the initial years. It should be noted that sufficient data were not available during this period. Over the years a lot of countries have joined this cluster of about 100% enrollment and 1-2 children. A negative relationship between fertility and secondary education, as well as gross national income per capita, is visible during the whole period. Nonetheless, this pattern is less significant for high-income economies.

In [11]:
# Scatter plot tertiary education
fig = px.scatter(tertiary_income, x='fertility', y='tertiary', animation_frame='year', animation_group='country',
        size='gni', color='region', hover_name='country',
        labels={
           'fertility': 'Fertility rate',
           'tertiary': 'School enrollment, tertiary, female %',
           'gni':'GNI per capita PPP'
        },
        title='Impact of tertiary education and income on fertility',
        range_x=[0.3,8], range_y=[0.3,145])

fig.add_annotation(dict(font=dict(size=13), x=0, y=-0.12, showarrow=False,
                                    text='Size is GNI per capita PPP',
                                    textangle=0,
                                    xanchor='left',
                                    xref='paper', yref='paper'))


fig.layout.updatemenus[0].buttons[0]['visible'] = False
fig.layout.updatemenus[0].buttons[1]['visible'] = False
fig.show()

Similar results with the previous plot are presented in this one. It also illustrates the substantial increase in the number of female students enrolling in tertiary education over the years.

# 2. The relationship of fertility with infant mortality and the Human Capital Index in 2017

Two other variables that may influence fertility are the infant mortality rate and the Human Capital Index (HCI):
* Infant mortality measures the number of infants that do not reach 1 year of age, per 1000 live births in a year.
* The HCI is a measure calculated by the World Bank, which evaluates the potential of a child as a future worker, taking into account the ideal health and education conditions. The index ranges from 0 to 1, where 1 denotes that the optimal conditions are met.

In [12]:
query2 = """

    SELECT  country_name, indicator_name, value, year, region
            FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` AS indices
            JOIN `bigquery-public-data.world_bank_health_population.country_summary` AS regions
                ON indices.country_code = regions.country_code
            WHERE indicator_code IN ('SP.DYN.TFRT.IN', 'SP.DYN.IMRT.IN', 'HD.HCI.OVRL') AND year=2017
            ORDER BY country_name, indicator_name, year
            ;
        """

In [13]:
mortality_hci = client.query(query2).result().to_dataframe()

In [14]:
# Dataframe for 3d plot

# Sort
mortality_hci = mortality_hci.sort_values(by=['region','country_name', 'indicator_name', 'year'])

# Pivot
mortality_hci = mortality_hci.pivot(values='value', index=['country_name','year','region'], 
                                                columns=['indicator_name'])

mortality_hci = mortality_hci.sort_values(by=['region','country_name', 'year'])
mortality_hci = mortality_hci.reset_index()

# Rename columns
mortality_hci.set_axis(['country', 'year', 'region', 'fertility', 'hci', 'mortality'], axis='columns', 
                                                                                         inplace=True)
# Cleansing
mortality_hci = mortality_hci.fillna(0)
mortality_hci = mortality_hci[mortality_hci['region']!=0]
mortality_hci = mortality_hci[mortality_hci['hci']!=0]
mortality_hci = mortality_hci[mortality_hci['fertility']!=0]

In [15]:
# 3d plot
fig = px.scatter_3d(mortality_hci, x='mortality', y='hci', z='fertility', color='region', hover_name='country',
        labels={
           'fertility': 'Fertility',
           'mortality': 'Infant mortality',
           'hci':'HCI'
        },
        title='The relationship of fertility with infant mortality and the Human Capital Index in 2017')

camera = dict(
    up=dict(x=0, y=2, z=1),
    center=dict(x=0, y=0, z=-0.27),
    eye=dict(x=-1.4, y=1.3, z=0.7)
)

fig.update_layout(scene_camera=camera)
fig.show()

According to this plot, a decreased Human Capital Index is connected to higher fertility rates in 2017. Furthermore, infant mortality has a positive relationship with fertility. There is a considerable increase in fertility rates when infant mortality surpasses a specific level around 20 and the HCI is less than 0.6.

# 3. Other determinants of fertility


Three other possible socioeconomic determinants of fertility will be investigated in this section. These are the following:

* Female unemployment rate as a percentage of the female labor force.
* Urban population as a percentage of the total population.
* Current expenditure in healthcare goods and services during a year expressed as a percentage of GDP.

In [16]:
query3 = """

    SELECT  country_name, indicator_name, value, region
            FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` AS indices
            JOIN `bigquery-public-data.world_bank_health_population.country_summary` AS regions
                ON indices.country_code = regions.country_code
            WHERE indicator_code IN ('SP.DYN.TFRT.IN', 'SP.URB.TOTL.IN.ZS', 'SL.UEM.TOTL.FE.ZS','SH.XPD.CHEX.GD.ZS') AND 
                  year=2017
            ORDER BY country_name, indicator_name, year
            ;
        """

In [17]:
other_determinants = client.query(query3).result().to_dataframe()

In [18]:
# Dataframe for scatter subplots

# Sort
other_determinants = other_determinants.sort_values(by=['region','country_name', 'indicator_name'])

# Pivot
other_determinants = other_determinants.pivot(values='value', index=['country_name','region'], 
                                                columns=['indicator_name'])

other_determinants = other_determinants.sort_values(by=['region','country_name'])
other_determinants = other_determinants.reset_index()

# Rename columns
other_determinants.set_axis(['country', 'region', 'Health expenditure', 'Fertility', 'Female unemployment', 'Urban population'],
                                                                             axis='columns', inplace=True)
# Cleansing
other_determinants = other_determinants.dropna()
other_determinants['Urban population'].replace(1,100, inplace = True)

In [19]:
# Scatter subplots
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig = make_subplots(rows=7, cols=3)


colors = ['#636EFA','#EF553B','#00CC96','#AB63FA','#FFA15A','#19D3F3','#FF6692']
indicator_names = ['Female unemployment','Urban population','Health expenditure']
i = 1
for region in other_determinants.region.unique():
    j = 1
    for indicator in indicator_names:
        fig.add_trace(go.Scatter(x=other_determinants[other_determinants['region']==region]['Fertility'],
                                 y=other_determinants[other_determinants['region']==region][indicator], 
                                 mode='markers', name=region, 
                                 hovertext=other_determinants[other_determinants['region']==region]['country'],  
                                 marker=dict(color=colors[i-1])),
                                 row=i, col=j)
        
        fig.update_xaxes(title_text='Fertility rate', row=i, col=j)
        fig.update_yaxes(title_text=indicator,  title_standoff = 0, row=i, col=j)
        j += 1
    i += 1

fig.update_layout(
    title_text='Other determinants of fertility (2017)',
    title_x=0.5,
    autosize = False,
    showlegend = True,
    width = 1000,
    height = 1500
),

# Remove unnecessary legend items
for i in range(1,21):
    if i not in range(3,21,3):
        fig['data'][i]['showlegend'] = False
    
fig.show()

The Middle East and North Africa countries exhibit a positive relationship between female unemployment and fertility rates. On the other hand, most of the European and Central Asia countries are clustered together, while for some of them a negative correlation exists.

Furthermore, a higher level of urbanization is connected to a lower fertility rate for most of the regions, except for Europe, where this relationship is weak.

A negative but not so strong correlation between current health expenditure and fertility exists for some regions such as Sub-Saharan Africa.

# 4. Fertility rates over time

In [20]:
query4 = """

    SELECT  country_name, indicator_name, value, year
            FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` AS indices
            WHERE indicator_code = 'SP.DYN.TFRT.IN' AND
                  country_code IN ('EAS', 'ECS', 'LCN', 'MEA', 'NAC', 'SAS', 'SSF', 'WLD')
            ORDER BY country_name, indicator_name, year
            ;
        """

In [21]:
fertility_region = client.query(query4).result().to_dataframe()

In [22]:
# Line chart
fig = px.line(fertility_region, x='year', y='value', color='country_name', hover_name='country_name', 
          hover_data={'country_name':False},
     labels={
           'value': 'Fertility rate',
           'year': 'Year',
           'country_name': 'Region',
        },
              
     title='Fertility rates by region, 1960-2018',
     color_discrete_map={'East Asia & Pacific':'#636EFA',
                         'Europe & Central Asia':'#EF553B',
                         'Latin America & Caribbean':'#00CC96',
                         'Middle East & North Africa':'#AB63FA',
                         'North America':'#FFA15A',
                         'South Asia':'#19D3F3',
                         'Sub-Saharan Africa':'#FF6692',
                         'World':'#585858'}
)

fig.show()

Τhe average number of children has been decreased from 5 to about 2.5 in this 58-year period. This decline was less pronounced during the last decades, and some regions such as Europe and Central Asia have also seen a mild increase in their rates.

In [23]:
query5 = """

    SELECT  country_name, indicator_name, value, year
            FROM `bigquery-public-data.world_bank_health_population.health_nutrition_population` AS indices
            WHERE indicator_code = 'SP.DYN.TFRT.IN' AND
                  country_code IN ('HIC', 'UMC', 'MIC', 'LMC', 'LIC')
            ORDER BY country_name, indicator_name, year
            ;
        """

In [24]:
fertility_income = client.query(query5).result().to_dataframe()

In [25]:
# Line chart
fig = px.line(fertility_income, x='year', y='value', color='country_name', hover_name='country_name',
            hover_data={'country_name' : False},
     labels={
           'value': 'Fertility rate',
           'year': 'Year',
           'country_name': 'Country income classification',
        },
     title='Fertility rates by country income level, 1960-2018',
     category_orders={'country_name': ['High income', 'Upper middle income','Middle income', 
                                                           'Lower middle income', 'Low income']},
     color_discrete_map={'High income':'#002266',
                         'Upper middle income':'#003cb3',
                         'Middle income':' #0055ff',
                         'Lower middle income':'#4d88ff',
                         'Low income':'#99bbff'}
)

fig.show()

As shown in the chart, there is a wide variation among the fertility rates of countries with different gross national income per capita. However, most of the countries have converged to a rate between 1.6 and 2.7 over the last years. The low-income countries are an exception, as their rate remained approximately 4.6.